In [ ]:
# 1) Runtime check and mount Google Drive
import sys, os
print('Python:', sys.version)
try:
    import torch
    print('Torch:', torch.__version__, 'CUDA available:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('Device:', torch.cuda.get_device_name(0))
except Exception as e:
    print('Torch import failed:', e)

from google.colab import drive
drive.mount('/content/drive')

## Upload files from your local machine
Use this cell to upload small files (CSVs, single images, or zips). For large datasets, keep files on Google Drive and use the Drive mount approach above.
- Upload single files or multiple files via the file picker.
- If you upload a ZIP of an image folder, the cell below will automatically unzip it into `data/Train` or `data/Validation`.

In [ ]:
# 1b) Upload files directly from your computer (small files)
from google.colab import files
uploaded = files.upload()  # opens file picker

import os, io, zipfile
# Default destination (change if needed)
train_dest = '/content/Image_Whitebalance_ML/data/Train'
val_dest = '/content/Image_Whitebalance_ML/data/Validation'
os.makedirs(train_dest, exist_ok=True)
os.makedirs(val_dest, exist_ok=True)

for fname, content in uploaded.items():
    # If zip, extract intelligently. Otherwise save to Train by default.
    lower = fname.lower()
    if lower.endswith('.zip'):
        # save zip then extract into Train folder (or specify)
        zpath = os.path.join(train_dest, fname)
        with open(zpath, 'wb') as f:
            f.write(content)
        try:
            with zipfile.ZipFile(zpath, 'r') as z:
                z.extractall(train_dest)
            print(f'Extracted zip to {train_dest}')
        except Exception as e:
            print('Error extracting zip:', e)
    else:
        # Save non-zip files to Train by default. If you want Validation, move after upload.
        dest = train_dest
        out_path = os.path.join(dest, fname)
        with open(out_path, 'wb') as f:
            f.write(content)
        print(f'Saved {fname} -> {out_path}')

print('
Upload complete. List /content/Image_Whitebalance_ML/data/Train (first 20 files):')
!ls -la /content/Image_Whitebalance_ML/data/Train | head -n 20

In [ ]:
# 2) Clone the repository into Colab workspace
!git clone https://github.com/shaktiaryan/Image_Whitebalance_ML.git /content/Image_Whitebalance_ML
%cd /content/Image_Whitebalance_ML
!git status --porcelain

In [ ]:
# 3) Install dependencies (use pip; adjust torch wheel for your CUDA if needed)
!pip install -r requirements.txt || true
# If torch is not installed or the version mismatches, install the matching wheel. Example (uncomment if needed):
# !pip install --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

In [ ]:
# 4) Optional: patch Windows paths in configs to Colab paths (safe operation)
cfg_file = 'configs/base_config.py'
import os
if os.path.exists(cfg_file):
    text = open(cfg_file, 'r', encoding='utf-8').read()
    text = text.replace('E:\\aftershoot', '/content/Image_Whitebalance_ML')
    open(cfg_file, 'w', encoding='utf-8').write(text)
    print('Patched', cfg_file)
else:
    print(cfg_file, 'not present or already OK')

In [ ]:
# 5) Prepare data: copy CSVs and link images from Google Drive (adjust your Drive path)
import os
os.makedirs('data/Train', exist_ok=True)
os.makedirs('data/Validation', exist_ok=True)
# Example: change these paths to where you stored dataset in Drive
DRIVE_ROOT = '/content/drive/MyDrive/aftershoot_data'  # <- update if needed
# Copy CSVs (fast)
if os.path.exists(os.path.join(DRIVE_ROOT, 'Train', 'sliders_filtered.csv')):
    !cp {DRIVE_ROOT}/Train/sliders_filtered.csv data/Train/sliders_filtered.csv
else:
    print('Train CSV not found in Drive path, please upload or change DRIVE_ROOT')
# Symlink images directory to avoid full copy (if available)
img_src = os.path.join(DRIVE_ROOT, 'Train', 'images')
img_dst = 'data/Train/images'
if os.path.exists(img_src) and not os.path.exists(img_dst):
    !ln -s {img_src} {img_dst}
    print('Created symlink to images')
else:
    print('Image source not found or destination exists')

In [ ]:
# 6) Run EDA (generates visuals under outputs/eda)
!python main.py --eda --config efficientnet

In [ ]:
# 7) Quick training smoke test (1 epoch)
!python main.py --config lightweight --epochs 1 --device cuda --output_dir outputs/colab_test_training

In [ ]:
# 8) Full training example (when ready)
# Uncomment and run when you have data and want a longer run
# !python main.py --config lightweight --epochs 15 --device cuda --output_dir outputs/colab_full_training

In [ ]:
# 9) Run model testing (after a training run finishes)
# !python test_model.py
!python test_colab_model.py || true

---
## Notes and caveats
- Use pip (not Poetry) in Colab. Poetry is optional.
- Keep large image data on Google Drive; symlink to avoid heavy copies.
- If timm attempts to download pretrained weights, allow the download or pre-download to Drive.
- If Colab runtime has no GPU, set `--device cpu`.
---
**Done**: run cells in order and check `outputs/*/logs/` and `outputs/*/logs/training_results.json`.